In [40]:
import cv2
import mediapipe as mp
import numpy as np
import os
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [61]:
def find_center(keypoints):
    # Calculate the center by taking the average of x and y coordinates
    center_x = np.mean([x for x, y in keypoints])
    center_y = np.mean([y for x, y in keypoints])
    return center_x, center_y

def normalize_distances(distances):
    max_distance = max(distances)
    min_distance = min(distances)
    
    if max_distance == min_distance:
        # Avoid division by zero if all distances are the same
        return [0.0] * len(distances)
    
    return [round((distance - min_distance) / (max_distance - min_distance) , 2) for distance in distances]



def calculate_distances_and_angles(keypoints, center):
    distances = []
    angles = []

    for x, y in keypoints:
        # Calculate Euclidean distance from the center
        distance = np.linalg.norm(np.array([x, y]) - np.array(center))
        distances.append(distance)

        # Calculate angle relative to the center (you may need to adjust this)
        angle = (np.arctan2(y - center[1], x - center[0]) * 180.0 / np.pi + 360) % 360
        angles.append(round(angle,1))
    
    normalized_distances = normalize_distances(distances)
    
    return normalized_distances, angles

In [53]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
#     if angle >180.0:
#         angle = 360-angle
    
    # Round to 1 decimal place
    angle = round(angle, 1)
    
        
    return angle 

In [24]:
import cv2

# Load image
image_path = "YogaIntelliJ/classification model/yoga_poses/train/tree/1.jpg"
image = cv2.imread(image_path)

# Resize image to 640x480
desired_size = (640, 480)
resized_image = cv2.resize(image, desired_size)

# Display the original and resized images
cv2.imshow('Original Image', image)
cv2.imshow('Resized Image', resized_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [89]:
import pandas as pd

# Sample data
data = {
    'angle_right_shoulder_hip_knee': [],
    'angle_right_hip_knee_ankle': [],
    'angle_left_shoulder_hip_knee': [],
    'angle_left_hip_knee_ankle': [],
    'angle_right_elbow_shoulder_hip': [],
    'angle_left_elbow_shoulder_hip': [],
    'angle_right_wrist_elbow_shoulder': [],
    'angle_left_wrist_elbow_shoulder': [],
    'ndright_shoulder': [],
    'aright_shoulder': [],
    'ndright_elbow': [],
    'aright_elbow': [],
    'ndright_wrist': [],
    'aright_wrist': [],
    'ndright_hip': [],
    'aright_hip': [],
    'ndright_knee': [],
    'aright_knee': [],
    'ndright_ankle': [],
    'aright_ankle': [],
    'ndleft_shoulder': [],
    'aleft_shoulder': [],
    'ndleft_elbow': [],
    'aleft_elbow': [],
    'ndleft_wrist': [],
    'aleft_wrist': [],
    'ndleft_hip': [],
    'aleft_hip': [],
    'ndleft_knee': [],
    'aleft_knee': [],
    'ndleft_ankle': [],
    'aleft_ankle': [],
    'pose':[]
}

# Create DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
print(df)


Empty DataFrame
Columns: [angle_right_shoulder_hip_knee, angle_right_hip_knee_ankle, angle_left_shoulder_hip_knee, angle_left_hip_knee_ankle, angle_right_elbow_shoulder_hip, angle_left_elbow_shoulder_hip, angle_right_wrist_elbow_shoulder, angle_left_wrist_elbow_shoulder, ndright_shoulder, aright_shoulder, ndright_elbow, aright_elbow, ndright_wrist, aright_wrist, ndright_hip, aright_hip, ndright_knee, aright_knee, ndright_ankle, aright_ankle, ndleft_shoulder, aleft_shoulder, ndleft_elbow, aleft_elbow, ndleft_wrist, aleft_wrist, ndleft_hip, aleft_hip, ndleft_knee, aleft_knee, ndleft_ankle, aleft_ankle, pose]
Index: []

[0 rows x 33 columns]


In [56]:
df

,angle_right_shoulder_hip_knee,angle_right_hip_knee_ankle,angle_left_shoulder_hip_knee,angle_left_hip_knee_ankle,angle_right_elbow_shoulder_hip,angle_left_elbow_shoulder_hip,angle_right_wrist_elbow_shoulder,angle_left_wrist_elbow_shoulder,ndright_shoulder,aright_shoulder,...,ndleft_elbow,aleft_elbow,ndleft_wrist,aleft_wrist,ndleft_hip,aleft_hip,ndleft_knee,aleft_knee,ndleft_ankle,aleft_ankle


In [255]:

# Load image
image_path = "YogaIntelliJ/classification model/yoga_poses/train/chair/girl1_chair070.jpg"
ori_image = cv2.imread(image_path)

# Resize image to 640x480
desired_size = (640, 480)
image = cv2.resize(ori_image, desired_size)


def PoseDetection(image):

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    # Recolor image to RGB
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Make detection
        results = pose.process(image_rgb)

    # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
        
        # Extract key landmarks for angle calculation
            nose = [landmarks[mp_pose.PoseLandmark.NOSE.value].x,
                landmarks[mp_pose.PoseLandmark.NOSE.value].y]

            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                       landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                       landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                     landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                      landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                       landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                      landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                      landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                    landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                      landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

            keypoints = [
                right_shoulder, right_elbow, right_wrist, right_hip, right_knee, right_ankle,
                left_shoulder, left_elbow, left_wrist, left_hip, left_knee, left_ankle
            ]

            center = nose
            normalized_distance, angles_from_center = calculate_distances_and_angles(keypoints, center)

            angle_right_shoulder_hip_knee = calculate_angle(right_shoulder, right_hip, right_knee)
            angle_right_hip_knee_ankle = calculate_angle(right_hip, right_knee, right_ankle)
            angle_left_shoulder_hip_knee = calculate_angle(left_shoulder, left_hip, left_knee)
            angle_left_hip_knee_ankle = calculate_angle(left_hip, left_knee, left_ankle)

            angle_right_elbow_shoulder_hip = calculate_angle(right_elbow, right_shoulder, right_hip)
            angle_left_elbow_shoulder_hip = calculate_angle(left_elbow, left_shoulder, left_hip)

            angle_right_wrist_elbow_shoulder = calculate_angle(right_wrist, right_elbow, right_shoulder)
            angle_left_wrist_elbow_shoulder = calculate_angle(left_wrist, left_elbow, left_shoulder)

        
        # Draw landmarks and connections on the image
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                   mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2, circle_radius=2),
                                   mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=2)
                                   )
        
            cv2.putText(image, str(angle_left_wrist_elbow_shoulder) + " , " + str(
            round(normalized_distance[7], 2)) + " , " + str(
            round(angles_from_center[7], 2)),
                    tuple(np.multiply(left_elbow, [640, 480]).astype(int)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

            cv2.putText(image, str(angle_right_wrist_elbow_shoulder) + " , " + str(
            round(normalized_distance[1], 2)) + " , " + str(
            round(angles_from_center[1], 2)),
                    tuple(np.multiply(right_elbow, [640, 480]).astype(int)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

            cv2.putText(image, str(angle_right_shoulder_hip_knee) + " , " + str(
            round(normalized_distance[3], 2)) + " , " + str(
            round(angles_from_center[3], 2)),
                    tuple(np.multiply(right_hip, [640, 480]).astype(int)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

            cv2.putText(image, str(angle_left_shoulder_hip_knee) + " , " + str(
            round(normalized_distance[9], 2)) + " , " + str(
            round(angles_from_center[9], 2)),
                    tuple(np.multiply(left_hip, [640, 480]).astype(int)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

            cv2.putText(image, str(angle_right_hip_knee_ankle) + " , " + str(
            round(normalized_distance[4], 2)) + " , " + str(
            round(angles_from_center[4], 2)),
                    tuple(np.multiply(right_knee, [640, 480]).astype(int)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

            cv2.putText(image, str(angle_left_hip_knee_ankle) + " , " + str(
            round(normalized_distance[10], 2)) + " , " + str(
            round(angles_from_center[10], 2)),
                    tuple(np.multiply(left_knee, [640, 480]).astype(int)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

            cv2.putText(image, str(angle_right_elbow_shoulder_hip) + " , " + str(
            round(normalized_distance[0], 2)) + " , " + str(
            round(angles_from_center[0], 2)),
                    tuple(np.multiply(right_shoulder, [640, 480]).astype(int)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

            cv2.putText(image, str(angle_left_elbow_shoulder_hip) + " , " + str(
            round(normalized_distance[6], 2)) + " , " + str(
            round(angles_from_center[6], 2)),
                    tuple(np.multiply(left_shoulder, [640, 480]).astype(int)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

            cv2.putText(image, " , " + str(round(normalized_distance[2], 2)) + " , " + str(
            round(angles_from_center[2], 2)),
                    tuple(np.multiply(right_wrist, [640, 480]).astype(int)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

            cv2.putText(image, " , " + str(round(normalized_distance[8], 2)) + " , " + str(
            round(angles_from_center[8], 2)),
                    tuple(np.multiply(left_wrist, [640, 480]).astype(int)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

            cv2.putText(image, " , " + str(round(normalized_distance[5], 2)) + " , " + str(
            round(angles_from_center[5], 2)),
                    tuple(np.multiply(right_ankle, [640, 480]).astype(int)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

            cv2.putText(image, " , " + str(round(normalized_distance[11], 2)) + " , " + str(
            round(angles_from_center[11], 2)),
                    tuple(np.multiply(left_ankle, [640, 480]).astype(int)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

        

#         center_x, center_y = tuple(np.multiply(nose, [640, 480]).astype(int))
#         cv2.circle(image, (center_x, center_y), 5, (0, 0, 255), -1)
                # Draw a red circle at the nose position
            center_x, center_y = tuple(np.multiply(nose, [640, 480]).astype(int))
            cv2.circle(image, (center_x, center_y), 5, (0, 0, 255), -1)


        # Display other information as needed...

        # Display the image with the drawn landmarks and information
#             cv2.imshow('Pose Detection Result', image)
#             cv2.waitKey(0)

        except Exception as e:
            print(f"An error occurred: {e}")

    cv2.destroyAllWindows()
    return{
    'angle_right_shoulder_hip_knee': angle_right_shoulder_hip_knee,
    'angle_right_hip_knee_ankle': angle_right_hip_knee_ankle,
    'angle_left_shoulder_hip_knee':angle_left_shoulder_hip_knee ,
    'angle_left_hip_knee_ankle': angle_left_hip_knee_ankle,
    'angle_right_elbow_shoulder_hip': angle_right_elbow_shoulder_hip,
    'angle_left_elbow_shoulder_hip': angle_left_elbow_shoulder_hip,
    'angle_right_wrist_elbow_shoulder': angle_right_wrist_elbow_shoulder,
    'angle_left_wrist_elbow_shoulder': angle_left_wrist_elbow_shoulder,
    'ndright_shoulder': normalized_distance[0],
    'aright_shoulder': angles_from_center[0],
    'ndright_elbow': normalized_distance[1],
    'aright_elbow': angles_from_center[1],
    'ndright_wrist': normalized_distance[2],
    'aright_wrist': angles_from_center[2],
    'ndright_hip': normalized_distance[3],
    'aright_hip': angles_from_center[3],
    'ndright_knee': normalized_distance[4],
    'aright_knee': angles_from_center[4],
    'ndright_ankle': normalized_distance[5],
    'aright_ankle': angles_from_center[5],
    'ndleft_shoulder': normalized_distance[6],
    'aleft_shoulder': angles_from_center[6],
    'ndleft_elbow': normalized_distance[7],
    'aleft_elbow': angles_from_center[7],
    'ndleft_wrist': normalized_distance[8],
    'aleft_wrist': angles_from_center[8],
    'ndleft_hip': normalized_distance[9],
    'aleft_hip': angles_from_center[9],
    'ndleft_knee': normalized_distance[10],
    'aleft_knee': angles_from_center[10],
    'ndleft_ankle': normalized_distance[11],
    'aleft_ankle': angles_from_center[11],
    'pose':0
}
PoseDetection(image)

{'angle_right_shoulder_hip_knee': 149.1,
 'angle_right_hip_knee_ankle': 234.1,
 'angle_left_shoulder_hip_knee': 154.9,
 'angle_left_hip_knee_ankle': 243.2,
 'angle_right_elbow_shoulder_hip': 174.6,
 'angle_left_elbow_shoulder_hip': 163.5,
 'angle_right_wrist_elbow_shoulder': 180.3,
 'angle_left_wrist_elbow_shoulder': 184.7,
 'ndright_shoulder': 0.09,
 'aright_shoulder': 141.6,
 'ndright_elbow': 0.06,
 'aright_elbow': 217.3,
 'ndright_wrist': 0.25,
 'aright_wrist': 252.9,
 'ndright_hip': 0.49,
 'aright_hip': 103.9,
 'ndright_knee': 0.76,
 'aright_knee': 87.9,
 'ndright_ankle': 1.0,
 'aright_ankle': 93.6,
 'ndleft_shoulder': 0.0,
 'aleft_shoulder': 102.8,
 'ndleft_elbow': 0.02,
 'aleft_elbow': 294.6,
 'ndleft_wrist': 0.22,
 'aleft_wrist': 288.4,
 'ndleft_hip': 0.46,
 'aleft_hip': 94.6,
 'ndleft_knee': 0.74,
 'aleft_knee': 85.2,
 'ndleft_ankle': 0.87,
 'aleft_ankle': 92.6,
 'pose': 0}

In [264]:
poses={
    0:'no_pose',
    1:'tree',
    2:'chair',
    3:'cobra',
    4:'shoulder_stand',
    5:'warrior',
    6:'triangle',
#     7:'dog',
    
}

In [256]:
# Set the path to the image directory
image_directory = "YogaIntelliJ/classification model/yoga_poses/train/no_pose/"

# Load all images from the directory
image_files = [f for f in os.listdir(image_directory) if os.path.isfile(os.path.join(image_directory, f))]
selected_images = image_files

In [257]:
len(selected_images)

26

In [224]:
selected_images.pop(389)

'guy2_cobra139_flipped.jpg'

In [258]:
import pandas as pd

# Assuming df is already defined as per your previous code

# List to store details for each image
all_details = []
count=1
# Loop through selected_images
for image_item in selected_images:
    # Load and resize image
    ori_image = cv2.imread("YogaIntelliJ/classification model/yoga_poses/train/no_pose/" + image_item)
    desired_size = (640, 480)
    image = cv2.resize(ori_image, desired_size)

    # Process image and get details
    details = PoseDetection(image)
    print(str(count) +":\n")
    print(details)
    print('\n')
    
    count+=1

    # Append details to the list
    all_details.append(details)

# Convert the list of dictionaries to a DataFrame
df_details = pd.DataFrame(all_details)

# # Concatenate df_details with the existing df
# df = pd.concat([df, df_details], ignore_index=True)

# # Display the updated DataFrame
# print(df)


1:

{'angle_right_shoulder_hip_knee': 185.6, 'angle_right_hip_knee_ankle': 180.3, 'angle_left_shoulder_hip_knee': 173.6, 'angle_left_hip_knee_ankle': 181.6, 'angle_right_elbow_shoulder_hip': 13.4, 'angle_left_elbow_shoulder_hip': 13.2, 'angle_right_wrist_elbow_shoulder': 181.3, 'angle_left_wrist_elbow_shoulder': 181.0, 'ndright_shoulder': 0.0, 'aright_shoulder': 122.9, 'ndright_elbow': 0.22, 'aright_elbow': 107.8, 'ndright_wrist': 0.44, 'aright_wrist': 103.9, 'ndright_hip': 0.37, 'aright_hip': 95.1, 'ndright_knee': 0.69, 'aright_knee': 92.2, 'ndright_ankle': 1.0, 'aright_ankle': 91.0, 'ndleft_shoulder': 0.01, 'aleft_shoulder': 50.2, 'ndleft_elbow': 0.21, 'aleft_elbow': 67.4, 'ndleft_wrist': 0.42, 'aleft_wrist': 73.4, 'ndleft_hip': 0.37, 'aleft_hip': 81.7, 'ndleft_knee': 0.7, 'aleft_knee': 84.9, 'ndleft_ankle': 1.0, 'aleft_ankle': 86.8, 'pose': 0}


2:

{'angle_right_shoulder_hip_knee': 193.1, 'angle_right_hip_knee_ankle': 171.2, 'angle_left_shoulder_hip_knee': 165.1, 'angle_left_hip_kn

11:

{'angle_right_shoulder_hip_knee': 183.0, 'angle_right_hip_knee_ankle': 176.0, 'angle_left_shoulder_hip_knee': 177.5, 'angle_left_hip_knee_ankle': 177.3, 'angle_right_elbow_shoulder_hip': 18.0, 'angle_left_elbow_shoulder_hip': 16.4, 'angle_right_wrist_elbow_shoulder': 173.9, 'angle_left_wrist_elbow_shoulder': 186.5, 'ndright_shoulder': 0.0, 'aright_shoulder': 110.6, 'ndright_elbow': 0.22, 'aright_elbow': 106.7, 'ndright_wrist': 0.43, 'aright_wrist': 103.4, 'ndright_hip': 0.35, 'aright_hip': 94.1, 'ndright_knee': 0.69, 'aright_knee': 91.8, 'ndright_ankle': 0.98, 'aright_ankle': 89.9, 'ndleft_shoulder': 0.01, 'aleft_shoulder': 56.0, 'ndleft_elbow': 0.23, 'aleft_elbow': 69.0, 'ndleft_wrist': 0.45, 'aleft_wrist': 75.3, 'ndleft_hip': 0.35, 'aleft_hip': 82.6, 'ndleft_knee': 0.7, 'aleft_knee': 87.3, 'ndleft_ankle': 1.0, 'aleft_ankle': 88.5, 'pose': 0}


12:

{'angle_right_shoulder_hip_knee': 185.1, 'angle_right_hip_knee_ankle': 178.8, 'angle_left_shoulder_hip_knee': 173.6, 'angle_left_hip

21:

{'angle_right_shoulder_hip_knee': 171.0, 'angle_right_hip_knee_ankle': 179.6, 'angle_left_shoulder_hip_knee': 184.1, 'angle_left_hip_knee_ankle': 185.7, 'angle_right_elbow_shoulder_hip': 11.8, 'angle_left_elbow_shoulder_hip': 13.2, 'angle_right_wrist_elbow_shoulder': 178.0, 'angle_left_wrist_elbow_shoulder': 163.6, 'ndright_shoulder': 0.0, 'aright_shoulder': 61.1, 'ndright_elbow': 0.23, 'aright_elbow': 74.7, 'ndright_wrist': 0.42, 'aright_wrist': 77.0, 'ndright_hip': 0.39, 'aright_hip': 86.0, 'ndright_knee': 0.69, 'aright_knee': 86.2, 'ndright_ankle': 1.0, 'aright_ankle': 86.1, 'ndleft_shoulder': 0.03, 'aleft_shoulder': 128.0, 'ndleft_elbow': 0.23, 'aleft_elbow': 110.5, 'ndleft_wrist': 0.37, 'aleft_wrist': 101.9, 'ndleft_hip': 0.39, 'aleft_hip': 96.7, 'ndleft_knee': 0.66, 'aleft_knee': 93.5, 'ndleft_ankle': 0.96, 'aleft_ankle': 93.2, 'pose': 0}


22:

{'angle_right_shoulder_hip_knee': 174.0, 'angle_right_hip_knee_ankle': 178.2, 'angle_left_shoulder_hip_knee': 189.2, 'angle_left_hi

In [262]:
df1=df.copy()

In [259]:
df = pd.concat([df, df_details], ignore_index=True)

In [81]:
img=cv2.imread(selected_images[261])
cv2.imshow("image",img)

error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:971: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'


In [263]:
df1

,angle_right_shoulder_hip_knee,angle_right_hip_knee_ankle,angle_left_shoulder_hip_knee,angle_left_hip_knee_ankle,angle_right_elbow_shoulder_hip,angle_left_elbow_shoulder_hip,angle_right_wrist_elbow_shoulder,angle_left_wrist_elbow_shoulder,ndright_shoulder,aright_shoulder,...,aleft_elbow,ndleft_wrist,aleft_wrist,ndleft_hip,aleft_hip,ndleft_knee,aleft_knee,ndleft_ankle,aleft_ankle,pose
0,215.1,25.6,178.3,179.0,13.2,19.4,42.7,41.7,0.00,119.8,...,70.8,0.09,86.4,0.39,87.2,0.70,90.3,1.00,91.4,1.0
1,181.6,180.3,145.3,25.0,19.0,12.4,38.2,42.8,0.01,118.1,...,71.6,0.10,85.5,0.38,83.8,0.62,75.8,0.50,86.7,1.0
2,229.2,64.2,177.0,179.1,22.0,24.6,54.4,53.2,0.00,118.0,...,68.0,0.12,87.1,0.40,87.3,0.71,90.3,1.00,91.6,1.0
3,184.0,180.4,132.3,292.4,25.0,20.0,48.9,55.0,0.00,119.0,...,68.6,0.13,85.5,0.37,83.6,0.59,72.2,0.63,86.5,1.0
4,242.1,26.0,173.4,186.4,18.2,26.4,33.1,30.1,0.01,131.6,...,62.7,0.02,83.5,0.40,85.6,0.70,88.3,1.00,91.3,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1545,174.0,178.2,189.2,177.4,10.9,9.9,191.1,182.0,0.02,52.6,...,104.0,0.44,101.7,0.40,94.2,0.69,94.3,1.00,93.7,0.0
1546,177.5,183.3,183.4,183.3,5.6,3.7,174.4,176.4,0.00,76.8,...,105.5,0.40,99.0,0.40,97.6,0.68,94.6,0.97,94.0,0.0
1547,176.0,178.4,179.7,178.1,10.4,6.0,191.9,182.7,0.04,54.6,...,96.2,0.43,96.3,0.39,90.8,0.71,89.6,1.00,88.5,0.0
1548,188.0,178.4,165.6,193.3,16.8,18.6,177.5,177.1,0.00,121.6,...,66.7,0.39,69.8,0.40,83.3,0.70,83.6,1.00,87.0,0.0


In [265]:
df.to_csv("pose_measures.csv",index=False)